In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import sys
csv.field_size_limit(sys.maxsize)

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

In [2]:
def load_df(csv_path='./data/train1000.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [column + '.' + subcolumn for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

In [34]:
JSON_COLUMNS_NEST = ['device', 'geoNetwork', 'totals', 'trafficSource']
JSON_COLUMNS_NEST_SINGLE = ['channelGrouping', 'visitNumber', 'visitStartNumber']

total_fea = ['pageviews', 'hits']
traffic_fea = ['medium','keyword', 'source', 'referralPath', 'isTrueDirect']
geo_fea = ['networkDomain','city','metro','region','country','continent','subContinent']
device_fea = ['deviceCategory','operatingSystem', 'browser']
fea = { 'device': device_fea, 'geoNetwork': geo_fea, 'totals': total_fea, 'trafficSource': traffic_fea}

csv_path='./data/train1000.csv'
extract_csv = './data/train_extract.csv'

with open(csv_path) as fp:  
    reader = csv.reader(fp)
    row1 = next(reader)
    df = pd.DataFrame(columns=row1)
    
    count = 0
    while(count < 10):
        missing=0
        dic = {}
        
        row = next(reader)
        row_standby = row.copy()
        d = json.dumps(row)
        g = json.loads(d)

        for col in row1:
            if col in JSON_COLUMNS_NEST:
                subobj = json.loads(row[row1.index(col)])
                for _, feat in enumerate(fea[col]):
                    try:
                        if subobj[feat] == 'not available in demo dataset' or subobj[feat] =='(not set)':
                            dic[str(col+ '.'+ feat)] = subobj[feat]
                            missing+=1
                    except KeyError as error:
                        missing+=1
        if missing < 10:
            df.loc[count] = row
            count+=1

In [56]:
print(dataframe.columns)
dataframe = dataframe.astype({"date": int, "visitId": int, "visitNumber": int, 'visitStartTime': int})
print(dataframe.dtypes)

Index(['channelGrouping', 'customDimensions', 'date', 'device',
       'fullVisitorId', 'geoNetwork', 'hits', 'socialEngagementType', 'totals',
       'trafficSource', 'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')
channelGrouping         object
customDimensions        object
date                     int64
device                  object
fullVisitorId           object
geoNetwork              object
hits                    object
socialEngagementType    object
totals                  object
trafficSource           object
visitId                  int64
visitNumber              int64
visitStartTime           int64
dtype: object
8934116514970143966
